In [1]:
##### Imports
import pandas as pd
import pysam
from Bio import Seq, SeqIO
from scripts import functions

In [2]:
##### Constants
REF_GENOME_DIR = 'data/hg38.gencode.v41.primary_genome_by_chromosome'
REF_ANNOTATIONS_FILE = 'data/hg38.gencode.v41.primary_annotation.gtf'

HG38_SAM = "data/gnomADv3.1.1/hg_38_inserts_x_introns.sam"

In [4]:
##### Settings
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', 200)

In [5]:
# ##### 
align_file = pysam.AlignmentFile(HG38_SAM, "r")
print(align_file.count())
print(align_file.has_index())
print(next(pysam.AlignmentFile(HG38_SAM, "r")))

441393
False
chrX|.|256940|insert_197	16	#1183124	3665	1	53M	*	0	0	GTTAACACAGTGAAACCCCGTCTCTACTAAAAATACAAAAAATTAGCCAGGCG	array('B', [40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40])	[('AS', 0), ('XS', 0), ('XN', 0), ('XM', 0), ('XO', 0), ('XG', 0), ('NM', 0), ('MD', '53'), ('YT', 'UU')]


In [6]:
##### 
matches = []
align_counts = {}
for ali in pysam.AlignmentFile(HG38_SAM):
	align_counts[ali.query_name] = align_counts.get(ali.query_name, 0) + 1

	if abs(ali.infer_query_length() - align_file.get_reference_length(ali.reference_name)) <= 5:
		matches.append(ali)
len(matches)

4

In [7]:
max(align_counts.values())

1

In [8]:
good_matches = []
for ali in matches:
	ref_name = ali.reference_name
	insert_chrom = ali.query_name.split("|")[0]
	intron_chrom = [item for item in ref_name.split("_") if "chr" in item][0]
	# print(ali)

	for insert in SeqIO.parse(f"data/gnomADv3.1.1/insert_fastas/gnomAD_v3.1.1_inserts_{insert_chrom}.fa", "fasta"):
		if insert.name == ali.query_name:
			insert_seq = str(insert.seq)
			assert len(insert_seq) == len(ali.query_sequence)
			break

	for intron in SeqIO.parse(f"data/hg38.ucsc_gencode_v41.introns_by_chromosome/{intron_chrom}.fa", "fasta"):
		if intron.name == ref_name:
			intron_seq = str(intron.seq)
			assert (len(intron_seq)) == align_file.get_reference_length(ali.reference_name)
			break
	stretcher_alignment = functions.emboss_stretcher(insert_seq, intron_seq, ali.query_name, ref_name)
	rev_alignment = functions.emboss_stretcher(Seq.reverse_complement(insert_seq), intron_seq, f"{ali.query_name}|rev", ref_name)
	if stretcher_alignment["Identity"] < rev_alignment["Identity"]:
		stretcher_alignment = rev_alignment
		# insert_seq = Seq.reverse_complement(insert_seq)
	stretcher_alignment["Insert_seq"] = insert_seq
	stretcher_alignment["Intron_seq"] = intron_seq
	stretcher_alignment["bowtie2_alignment"] = ali
	if stretcher_alignment["Identity"] >= 80:
		good_matches.append(stretcher_alignment)

print(len(good_matches))
good_matches

4


[{'Seq_1_name': 'chrX|.|6533768|insert_7223',
  'Seq_2_name': 'ENST00000398729.1_intron_2_0_chrX_6533889_r|6533889|6533948|-',
  'Length': 60,
  'Identity': 100.0,
  'Similarity': 100.0,
  'Gaps': 0,
  'Score': 300.0,
  'Display': 'chrX|.|653376      1 CTCGCTCTCCTGACTCAGTGGTTCCTCCACCTGGCTCTCCTGACTCAGTG     50\n                     ||||||||||||||||||||||||||||||||||||||||||||||||||\nENST000003987      1 CTCGCTCTCCTGACTCAGTGGTTCCTCCACCTGGCTCTCCTGACTCAGTG     50\n\nchrX|.|653376     51 GTTCTTCCAC     60\n                     ||||||||||\nENST000003987     51 GTTCTTCCAC     60',
  'Seq_1_align_start': 1,
  'Seq_1_align_end': 60,
  'Seq_2_align_start': 1,
  'Seq_2_align_end': 60,
  'Insert_seq': 'CTCGCTCTCCTGACTCAGTGGTTCCTCCACCTGGCTCTCCTGACTCAGTGGTTCTTCCAC',
  'Intron_seq': 'CTCGCTCTCCTGACTCAGTGGTTCCTCCACCTGGCTCTCCTGACTCAGTGGTTCTTCCAC',
  'bowtie2_alignment': <pysam.libcalignedsegment.AlignedSegment at 0x7f36dfd27d60>},
 {'Seq_1_name': 'chrX|rs1569057212|6533918|insert_7231|rev',
  'Seq_2_na

In [9]:
good_matches_df = {"Insert_chrom": [],
				"Intron_chrom": [],
				"Insert_start": [],
				"Intron_start": [],
				"Intron_end": [],
				"Intron_strand": [],
				# "Cigar_string": [],
				"Identity": [],
				"Gaps": [],
				"Insert_length": [],
				"Intron_length": [],
				"Insert_seq": [],
				"Intron_seq": [],
				"Insert_name": [],
				"Intron_name": [],
				}
for match in good_matches:
	good_matches_df["Insert_chrom"].append(match["Seq_1_name"].split("|")[0])
	good_matches_df["Intron_chrom"].append([item for item in match["Seq_2_name"].split("_") if "chr" in item][0])
	good_matches_df["Insert_start"].append(match["Seq_1_name"].split("|")[2])
	good_matches_df["Intron_start"].append(match["Seq_2_name"].split("|")[1])
	good_matches_df["Intron_end"].append(match["Seq_2_name"].split("|")[2])
	good_matches_df["Intron_strand"].append(match["Seq_2_name"].split("|")[3])
	# cigar = "M ".join(match["bowtie2_alignment"].cigarstring.split("M"))
	# good_matches_df["Cigar_string"].append("I ".join(cigar.split("I")))
	good_matches_df["Identity"].append(match["Identity"])
	good_matches_df["Gaps"].append(match["Gaps"])
	good_matches_df["Insert_length"].append(len(match["Insert_seq"]))
	good_matches_df["Intron_length"].append(len(match["Intron_seq"]))
	good_matches_df["Insert_seq"].append(match["Insert_seq"])
	good_matches_df["Intron_seq"].append(match["Intron_seq"])
	good_matches_df["Insert_name"].append(match["Seq_1_name"])
	good_matches_df["Intron_name"].append(match["Seq_2_name"])
good_matches_df = pd.DataFrame(good_matches_df)
good_matches_df.sort_values(by=["Insert_chrom", "Intron_chrom", "Insert_start", "Intron_start"], inplace=True, ignore_index=True)
good_matches_df

,Insert_chrom,Intron_chrom,Insert_start,Intron_start,Intron_end,Intron_strand,Identity,Gaps,Insert_length,Intron_length,Insert_seq,Intron_seq,Insert_name,Intron_name
0,chr3,chr3,50117173,50117172,50117249,+,96.2,3,77,78,ATGTGATGTGCACATTTTCCAGTTCGTAAGCTGGGGCCCTGGCTGT...,GTATGTGATGTGCACATTTTCCAGTTCGTAAGCTGGGGCCCTGGCT...,chr3|.|50117173|insert_14282,ENST00000347869.8_intron_22_0_chr3_50117172_f|...
1,chrX,chrX,6533768,6533889,6533948,-,100.0,0,60,60,CTCGCTCTCCTGACTCAGTGGTTCCTCCACCTGGCTCTCCTGACTC...,CTCGCTCTCCTGACTCAGTGGTTCCTCCACCTGGCTCTCCTGACTC...,chrX|.|6533768|insert_7223,ENST00000398729.1_intron_2_0_chrX_6533889_r|65...
2,chrX,chrX,6533918,7843903,7843962,+,96.7,0,60,60,CTGGCTCTCCTGACTCAGTGGTTCTTCCACCTCGCTCTCCTGACTC...,GTGGAGGAACCACTGAGTCAGGAGAGCGAGATGGAAGAACCACTGA...,chrX|rs1569057212|6533918|insert_7231|rev,ENST00000620630.2_intron_1_0_chrX_7843903_f|78...
3,chrX,chrX,7843938,7843873,7843992,+,88.1,12,120,120,GAACCACTGAGTCAGGAGAGCCAGGTGGAGGAACCACCGAGTCAGG...,GTGGAGGAACCACTGAGTCAGGAGAGCCAGGTGGAGGAACCACTGA...,chrX|.|7843938|insert_7487,ENST00000692567.1_intron_1_0_chrX_7843873_f|78...


In [13]:
for _, row in good_matches_df.iterrows():
	if row["Intron_name"].startswith("ENST00000398729.1_intron_2_0_chrX_6533889_r"):
		print(row["Insert_seq"], "\t\t\t", Seq.reverse_complement(row["Intron_seq"]))
	else:
		print(row["Insert_seq"], "\t\t\t", row["Intron_seq"])

ATGTGATGTGCACATTTTCCAGTTCGTAAGCTGGGGCCCTGGCTGTTTTAAGTAACTGTGTGTTTGCCACTGGCAGG 			 GTATGTGATGTGCACATTTTCCAGTTCGTAAGCTGGGGCCCTGGCTGTTTTAAGTAACTGTGTGTTTGCCACTGGCAG
CTCGCTCTCCTGACTCAGTGGTTCCTCCACCTGGCTCTCCTGACTCAGTGGTTCTTCCAC 			 GTGGAAGAACCACTGAGTCAGGAGAGCCAGGTGGAGGAACCACTGAGTCAGGAGAGCGAG
CTGGCTCTCCTGACTCAGTGGTTCTTCCACCTCGCTCTCCTGACTCAGTGGTTCCTCCAG 			 GTGGAGGAACCACTGAGTCAGGAGAGCGAGATGGAAGAACCACTGAGTCAGGAGAGCCAG
GAACCACTGAGTCAGGAGAGCCAGGTGGAGGAACCACCGAGTCAGGAGAGCGAGATGGAGGAACCACTGAGTCAGGAGAGCCAGGTGGAGGAACCACTGAGTCAGGAGAGCGAGATGGAG 			 GTGGAGGAACCACTGAGTCAGGAGAGCCAGGTGGAGGAACCACTGAGTCAGGAGAGCGAGATGGAAGAACCACTGAGTCAGGAGAGCCAGGTGGAGGAACCACCGAGTCAGGAGAGCGAG


In [5]:
Seq.reverse_complement("CTCGCTCTCCTGACTCAGTGGTTCCTCCACCTGGCTCTCCTGACTCAGTGGTTCTTCCAC")

'GTGGAAGAACCACTGAGTCAGGAGAGCCAGGTGGAGGAACCACTGAGTCAGGAGAGCGAG'

In [2]:
'GTGGAAGAACCACTGAGTCAGGAGAGCCAGGTGGAGGAACCACTGAGTCAGGAGAGCGAG' == Seq.reverse_complement('CTCGCTCTCCTGACTCAGTGGTTCCTCCACCTGGCTCTCCTGACTCAGTGGTTCTTCCAC')

True

In [ ]:
# ATGTGATGTG
# CACATTTTCC
# AGTTCGTAAG
# CTGGGGCCCT
# GGCTGTTTTA
# AGTAACTGTG
# TGTTTGCCAC
# TGGCAGG

# GTATGTGATG
# TGCACATTTT
# CCAGTTCGTA
# AGCTGGGGCC
# CTGGCTGTTT
# TAAGTAACTG
# TGTGTTTGCC
# ACTGGCAG


# CTCGCTCTCC
# TGACTCAGTG
# GTTCCTCCAC
# CTGGCTCTCC
# TGACTCAGTG
# GTTCTTCCAC

# GTGGAAGAAC
# CACTGAGTCA
# GGAGAGCCAG
# GTGGAGGAAC
# CACTGAGTCA
# GGAGAGCGAG


# CTGGCTCTCC
# TGACTCAGTG
# GTTCTTCCAC
# CTCGCTCTCC
# TGACTCAGTG
# GTTCCTCCAG

# GTGGAGGAAC
# CACTGAGTCA
# GGAGAGCGAG
# ATGGAAGAAC
# CACTGAGTCA
# GGAGAGCCAG


# GAACCACTGA
# GTCAGGAGAG
# CCAGGTGGAG
# GAACCACCGA
# GTCAGGAGAG
# CGAGATGGAG
# GAACCACTGA
# GTCAGGAGAG
# CCAGGTGGAG
# GAACCACTGA
# GTCAGGAGAG
# CGAGATGGAG

# GTGGAGGAAC
# CACTGAGTCA
# GGAGAGCCAG
# GTGGAGGAAC
# CACTGAGTCA
# GGAGAGCGAG
# ATGGAAGAAC
# CACTGAGTCA
# GGAGAGCCAG
# GTGGAGGAAC
# CACCGAGTCA
# GGAGAGCGAG